# 🧠 QuanSkill's Personal Study Copilot – Assignment 01 Notebook (Open‑Source Version)
# 🧠 QuanSkill's Personal Study Copilot – Assignment 01 Notebook (Phiên bản Mã nguồn mở)

This is your **guided assignment notebook** for building a _Personal Study Copilot_ agent.
Đây là **notebook hướng dẫn** để xây dựng một agent _Personal Study Copilot_ (Trợ lý học tập cá nhân).

In this version, we **do not use any paid API keys**. Everything runs on **open‑source models**:
Trong phiên bản này, chúng ta **không sử dụng bất kỳ API key trả phí nào**. Mọi thứ chạy trên **các mô hình mã nguồn mở**:

- Text embeddings from `sentence-transformers/all-MiniLM-L6-v2`
- Embeddings văn bản từ `sentence-transformers/all-MiniLM-L6-v2`
- A small open‑source chat model via `transformers` + `HuggingFacePipeline` (e.g., TinyLlama)
- Mô hình chat mã nguồn mở nhỏ qua `transformers` + `HuggingFacePipeline` (ví dụ: TinyLlama)

You will:
Bạn sẽ:

- Ingest study documents
- Nạp các tài liệu học tập
- Build a vector store (FAISS)
- Xây dựng kho lưu trữ vector (FAISS)
- Create a RAG chain using LCEL
- Tạo chuỗi RAG sử dụng LCEL
- Add simple conversational memory
- Thêm bộ nhớ hội thoại đơn giản
- Build a tool (calculator or similar)
- Xây dựng một công cụ (máy tính hoặc tương tự)
- Use LangGraph to route between RAG and the tool
- Sử dụng LangGraph để định tuyến giữa RAG và công cụ
- Expose a `chat_loop()` to talk to your agent
- Tạo một `chat_loop()` để trò chuyện với agent của bạn

> 🔧 This notebook contains explanations + **TODOs**. Your job is to fill them in.
> 🔧 Notebook này chứa các giải thích + **TODOs**. Nhiệm vụ của bạn là điền vào chúng.


## ✅ High-Level Checklist
## ✅ Danh sách kiểm tra tổng quan

By the end, your notebook should:
Đến cuối notebook, bạn nên có:

1. Ingest `.txt` / `.md` documents into a vector store.
1. Nạp các tài liệu `.txt` / `.md` vào kho lưu trữ vector.
2. Implement `retrieve_context(query: str) -> List[str]`.
2. Triển khai `retrieve_context(query: str) -> List[str]`.
3. Implement a LCEL-style RAG chain + `answer_with_rag(query: str)`.
3. Triển khai chuỗi RAG kiểu LCEL + `answer_with_rag(query: str)`.
4. Maintain some conversation memory.
4. Duy trì bộ nhớ hội thoại.
5. Use LangGraph with:
5. Sử dụng LangGraph với:
   - `router` node (decides RAG vs Tool)
   - Nút `router` (quyết định RAG hay Tool)
   - `rag_node`
   - Nút `rag_node`
   - `tool_node` (calculator or other)
   - Nút `tool_node` (máy tính hoặc công cụ khác)
6. Provide a `chat_loop()` to interact with the agent.
6. Cung cấp `chat_loop()` để tương tác với agent.


## 0. Environment Setup (Open‑Source Stack)
## 0. Thiết lập môi trường (Ngăn xếp mã nguồn mở)

We will use these packages:
Chúng ta sẽ sử dụng các gói sau:

- `langchain`, `langchain-community` – core + community integrations
- `langchain`, `langchain-community` – lõi + tích hợp cộng đồng
- `langgraph` – to define the agent graph
- `langgraph` – để định nghĩa đồ thị agent
- `faiss-cpu` – vector store backend
- `faiss-cpu` – backend kho lưu trữ vector
- `transformers`, `accelerate` – to run open‑source LLMs
- `transformers`, `accelerate` – để chạy các LLM mã nguồn mở
- `sentence-transformers` – for embeddings
- `sentence-transformers` – cho embeddings

Run the cell below once to install them if needed.
Chạy cell bên dưới một lần để cài đặt chúng nếu cần.


In [ ]:
# ⚙️ Install dependencies (uncomment if needed)
# ⚙️ Cài đặt các phụ thuộc (bỏ comment nếu cần)

# !pip install -q langchain langchain-community langgraph
# !pip install -q faiss-cpu
# !pip install -q transformers accelerate sentence-transformers


## 1. Imports & Basic Configuration
## 1. Import và Cấu hình cơ bản

Here we import everything we need for the assignment.
Ở đây chúng ta import mọi thứ cần thiết cho bài tập.
You may remove imports later if you don't use them.
Bạn có thể xóa các import sau này nếu không sử dụng chúng.


In [ ]:
from typing import List, Dict, Any
from dataclasses import dataclass, field
import os
import math
import re

# LangChain core
# Lõi LangChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda

# Vector store + embeddings (open‑source)
# Kho lưu trữ vector + embeddings (mã nguồn mở)
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFacePipeline

# Text splitter
# Bộ chia văn bản
from langchain_text_splitters import RecursiveCharacterTextSplitter

# LangGraph
from langgraph.graph import StateGraph, END

# Transformers for open‑source LLM
# Transformers cho LLM mã nguồn mở
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


## 2. Agent State (LangGraph)
## 2. Trạng thái Agent (LangGraph)

We define a simple state class to track what flows through the agent graph.
Chúng ta định nghĩa một lớp trạng thái đơn giản để theo dõi những gì chảy qua đồ thị agent.
You can extend it if needed, but keep these fields at minimum.
Bạn có thể mở rộng nó nếu cần, nhưng giữ tối thiểu các trường này.


In [ ]:
@dataclass
class AgentState:
    user_input: str  # Đầu vào từ người dùng
    messages: List[Dict[str, Any]] = field(default_factory=list)  # Danh sách tin nhắn
    route: str = ''          # 'rag' or 'tool' - 'rag' hoặc 'tool'
    tool_result: Any = None  # output from tool - Kết quả từ công cụ


## 3. Document Ingestion & Vector Store
## 3. Nạp tài liệu và Kho lưu trữ Vector

Steps:
Các bước:

1. Point `DOCS_FOLDER` to a directory with `.txt` / `.md` study files.
1. Trỏ `DOCS_FOLDER` đến thư mục chứa các file học tập `.txt` / `.md`.
2. Load all files and read them as raw strings.
2. Tải tất cả các file và đọc chúng dưới dạng chuỗi thô.
3. Split the raw texts into chunks using `RecursiveCharacterTextSplitter`.
3. Chia các văn bản thô thành các đoạn bằng `RecursiveCharacterTextSplitter`.
4. Use `HuggingFaceEmbeddings` + `FAISS.from_documents` to build a vector store.
4. Sử dụng `HuggingFaceEmbeddings` + `FAISS.from_documents` để xây dựng kho lưu trữ vector.

> ✅ At the end, you should have a global `vectorstore` object.
> ✅ Cuối cùng, bạn nên có một đối tượng `vectorstore` toàn cục.


In [ ]:
# TODO: change this path if needed (e.g., '/content/docs' in Colab)
# TODO: thay đổi đường dẫn này nếu cần (ví dụ: '/content/docs' trong Colab)
DOCS_FOLDER = 'data/docs'

os.makedirs(DOCS_FOLDER, exist_ok=True)

def load_raw_documents(docs_folder: str) -> List[str]:
    # Load all .txt and .md files from docs_folder as a list of strings.
    # Tải tất cả các file .txt và .md từ docs_folder dưới dạng danh sách chuỗi.
    texts = []
    for fname in os.listdir(docs_folder):
        fpath = os.path.join(docs_folder, fname)
        if not os.path.isfile(fpath):
            continue
        if fname.lower().endswith(('.txt', '.md')):
            with open(fpath, 'r', encoding='utf-8', errors='ignore') as f:
                texts.append(f.read())
    return texts

raw_docs = load_raw_documents(DOCS_FOLDER)
print(f'Loaded {len(raw_docs)} raw documents.')


In [ ]:
# Split documents into chunks
# Chia tài liệu thành các đoạn

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Kích thước mỗi đoạn: 1000 ký tự
    chunk_overlap=200,    # Độ chồng lấp giữa các đoạn: 200 ký tự
    add_start_index=True, # Thêm chỉ số bắt đầu
)

all_splits = text_splitter.create_documents(raw_docs)
print(f'Created {len(all_splits)} text chunks.')


In [ ]:
# TODO: Build vectorstore using open‑source embeddings (HuggingFaceEmbeddings) + FAISS
# TODO: Xây dựng vectorstore sử dụng embeddings mã nguồn mở (HuggingFaceEmbeddings) + FAISS

# Example (you may keep or modify):
# Ví dụ (bạn có thể giữ hoặc sửa đổi):
# embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
# vectorstore = FAISS.from_documents(all_splits, embeddings)

vectorstore = None  # 🔴 TODO: replace with real vectorstore - 🔴 TODO: thay thế bằng vectorstore thật

if vectorstore is None:
    print('⚠️ vectorstore is still None – implement it above.')
    print('⚠️ vectorstore vẫn là None – hãy triển khai nó ở trên.')
else:
    print('✅ Vector store created.')
    print('✅ Đã tạo kho lưu trữ vector.')


## 4. Retriever – `retrieve_context`
## 4. Retriever – `retrieve_context`

Once `vectorstore` is ready, create a retriever and implement:
Khi `vectorstore` đã sẵn sàng, tạo một retriever và triển khai:

`retrieve_context(query: str, k: int = 4) -> List[str]`

This should return a list of relevant text chunks.
Hàm này nên trả về danh sách các đoạn văn bản liên quan.


In [ ]:
# TODO: create retriever from vectorstore
# TODO: tạo retriever từ vectorstore
retriever = None  # e.g., vectorstore.as_retriever(search_kwargs={'k': 4})
                  # ví dụ: vectorstore.as_retriever(search_kwargs={'k': 4})

def retrieve_context(query: str, k: int = 4) -> List[str]:
    # Return top-k relevant text chunks for the query using retriever.
    # Trả về top-k đoạn văn bản liên quan cho truy vấn sử dụng retriever.
    if retriever is None:
        raise ValueError('retriever is not initialized – create it from vectorstore.')
        raise ValueError('retriever chưa được khởi tạo – hãy tạo nó từ vectorstore.')
    docs = retriever.get_relevant_documents(query)
    return [d.page_content for d in docs]

# You can test this later after building the vectorstore and retriever.
# Bạn có thể kiểm tra điều này sau khi xây dựng vectorstore và retriever.


## 5. Open‑Source LLM + RAG Chain
## 5. LLM mã nguồn mở + Chuỗi RAG

Now we set up:
Bây giờ chúng ta thiết lập:

1. An open‑source LLM via `transformers` + `HuggingFacePipeline`.
1. Một LLM mã nguồn mở qua `transformers` + `HuggingFacePipeline`.
2. A `ChatPromptTemplate` for RAG.
2. Một `ChatPromptTemplate` cho RAG.
3. A LCEL chain: `lookup_context -> prompt -> llm`.
3. Một chuỗi LCEL: `lookup_context -> prompt -> llm`.
4. A helper function `answer_with_rag(query: str) -> str`.
4. Một hàm helper `answer_with_rag(query: str) -> str`.

Suggested small chat model (no key needed):
Mô hình chat nhỏ được đề xuất (không cần key):

- `TinyLlama/TinyLlama-1.1B-Chat-v1.0`

You may change to a different model, but keep it reasonably small.
Bạn có thể đổi sang mô hình khác, nhưng hãy giữ nó ở kích thước hợp lý.


In [ ]:
# TODO: Initialize open‑source LLM using transformers + HuggingFacePipeline
# TODO: Khởi tạo LLM mã nguồn mở sử dụng transformers + HuggingFacePipeline

# Example (you can adapt hyperparameters as needed):
# Ví dụ (bạn có thể điều chỉnh các siêu tham số nếu cần):
# model_name = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)
# gen_pipe = pipeline('text-generation', model=model, tokenizer=tokenizer,
#                     max_new_tokens=256, do_sample=False)
# llm = HuggingFacePipeline(pipeline=gen_pipe)

llm = None  # 🔴 TODO: replace with real HuggingFacePipeline-based llm
            # 🔴 TODO: thay thế bằng llm dựa trên HuggingFacePipeline thật


In [ ]:
# Prompt template for RAG
# Mẫu prompt cho RAG
rag_prompt = ChatPromptTemplate.from_template(
    'You are a helpful study assistant.\n\n'
    'Use ONLY the following context to answer the question.\n'
    'If the answer is not in the context, say: "I do not see this in the study materials."\n\n'
    'Context:\n{context}\n\n'
    'Question:\n{question}\n\n'
    'Give a clear and concise answer.'
)

def _lookup_context(inputs: Dict[str, Any]) -> Dict[str, Any]:
    # Hàm tra cứu ngữ cảnh từ retriever
    query = inputs['question']
    chunks = retrieve_context(query)
    return {
        'context': '\n\n'.join(chunks),
        'question': query,
    }

# TODO: Build LCEL RAG chain only after llm is initialized
# TODO: Xây dựng chuỗi RAG LCEL chỉ sau khi llm được khởi tạo
lookup = RunnableLambda(_lookup_context)
rag_chain = None  # 🔴 TODO: set to lookup | rag_prompt | llm
                  # 🔴 TODO: đặt thành lookup | rag_prompt | llm

def answer_with_rag(query: str) -> str:
    # Use retrieval + llm to answer a question.
    # Sử dụng retrieval + llm để trả lời câu hỏi.
    if rag_chain is None:
        raise ValueError('rag_chain is not initialized – build it after llm is ready.')
        raise ValueError('rag_chain chưa được khởi tạo – hãy xây dựng nó sau khi llm sẵn sàng.')
    result = rag_chain.invoke({'question': query})
    # HuggingFacePipeline returns a string or generation object depending on config.
    # HuggingFacePipeline trả về một chuỗi hoặc đối tượng generation tùy theo cấu hình.
    return str(result)


## 6. Conversational Memory (Simple)
## 6. Bộ nhớ hội thoại (Đơn giản)

We use a simple Python list as memory:
Chúng ta sử dụng một danh sách Python đơn giản làm bộ nhớ:

- `remember_message(role, content)`
- `remember_message(role, content)` - Ghi nhớ tin nhắn
- `get_formatted_history()` (optional, if you want to inject history into prompts later)
- `get_formatted_history()` (tùy chọn, nếu bạn muốn chèn lịch sử vào prompts sau này)


In [ ]:
# Lưu trữ lịch sử hội thoại
conversation_history: List[Dict[str, str]] = []

def remember_message(role: str, content: str):
    # Ghi nhớ một tin nhắn vào lịch sử hội thoại
    conversation_history.append({'role': role, 'content': content})

def get_formatted_history() -> str:
    # Lấy lịch sử hội thoại đã định dạng
    lines = []
    for msg in conversation_history:
        lines.append(f"{msg['role'].upper()}: {msg['content']}")
    return '\n'.join(lines)


## 7. Tool Implementation (Calculator Example)
## 7. Triển khai công cụ (Ví dụ máy tính)

Implement at least one tool. Here we suggest a **calculator**:
Triển khai ít nhất một công cụ. Ở đây chúng ta đề xuất một **máy tính**:

- Input: a math expression string (e.g., `2 + 3 * 4`).
- Đầu vào: một chuỗi biểu thức toán học (ví dụ: `2 + 3 * 4`).
- Output: a safe evaluation result.
- Đầu ra: kết quả đánh giá an toàn.

> ⚠️ Keep it simple and safe; do not expose full `eval` in real systems.
> ⚠️ Giữ nó đơn giản và an toàn; không để lộ toàn bộ `eval` trong hệ thống thực tế.


In [ ]:
def calculator_tool(expression: str) -> str:
    # Evaluate a simple math expression like '2 + 3 * 4'.
    # Đánh giá một biểu thức toán học đơn giản như '2 + 3 * 4'.
    try:
        # Chỉ cho phép sử dụng các hàm từ module math để đảm bảo an toàn
        allowed_names = {k: v for k, v in math.__dict__.items() if not k.startswith('__')}
        result = eval(expression, {'__builtins__': {}}, allowed_names)
        return f'Result: {result}'
    except Exception as e:
        return f'Error evaluating expression: {e}'


## 8. Router Node – Decide RAG vs Tool
## 8. Nút Router – Quyết định RAG hay Tool

We implement a simple heuristic router:
Chúng ta triển khai một router heuristic đơn giản:

- If the query contains digits AND math symbols → route to `tool`.
- Nếu truy vấn chứa chữ số VÀ ký hiệu toán học → định tuyến đến `tool`.
- Otherwise → route to `rag`.
- Ngược lại → định tuyến đến `rag`.


In [ ]:
def router_node(state: AgentState) -> AgentState:
    # Nút router quyết định điều hướng đến RAG hay Tool
    query = state.user_input.strip()
    # Kiểm tra nếu truy vấn chứa số và ký hiệu toán học
    if re.search(r'[0-9]', query) and re.search(r'[+\-*/]', query):
        state.route = 'tool'
    else:
        state.route = 'rag'
    return state


## 9. RAG Node – Handle Study Queries
## 9. Nút RAG – Xử lý truy vấn học tập

This node will:
Nút này sẽ:

1. Log the user query to memory.
1. Ghi lại truy vấn người dùng vào bộ nhớ.
2. Call `answer_with_rag`.
2. Gọi `answer_with_rag`.
3. Log the answer.
3. Ghi lại câu trả lời.
4. Append both to `state.messages`.
4. Thêm cả hai vào `state.messages`.
5. Print the answer.
5. In câu trả lời.


In [ ]:
def rag_node(state: AgentState) -> AgentState:
    # Nút RAG xử lý các truy vấn học tập
    query = state.user_input
    remember_message('user', query)  # Ghi nhớ câu hỏi của người dùng
    answer = answer_with_rag(query)  # Trả lời sử dụng RAG
    remember_message('assistant', answer)  # Ghi nhớ câu trả lời
    state.messages.append({'role': 'user', 'content': query})
    state.messages.append({'role': 'assistant', 'content': answer})
    print(f'Agent (RAG): {answer}')
    return state


## 10. Tool Node – Handle Utility Queries
## 10. Nút Tool – Xử lý truy vấn tiện ích

This node will:
Nút này sẽ:

1. Log the user query.
1. Ghi lại truy vấn người dùng.
2. Call `calculator_tool`.
2. Gọi `calculator_tool`.
3. Store result in `state.tool_result`.
3. Lưu kết quả vào `state.tool_result`.
4. Log answer + print it.
4. Ghi lại câu trả lời + in nó.


In [ ]:
def tool_node(state: AgentState) -> AgentState:
    # Nút Tool xử lý các truy vấn tiện ích (ví dụ: tính toán)
    query = state.user_input
    remember_message('user', query)  # Ghi nhớ câu hỏi của người dùng
    tool_output = calculator_tool(query)  # Gọi công cụ máy tính
    state.tool_result = tool_output  # Lưu kết quả vào state
    remember_message('assistant', tool_output)  # Ghi nhớ kết quả
    state.messages.append({'role': 'user', 'content': query})
    state.messages.append({'role': 'assistant', 'content': tool_output})
    print(f'Agent (TOOL): {tool_output}')
    return state


## 11. Build the LangGraph
## 11. Xây dựng LangGraph

We wire the graph:
Chúng ta kết nối đồ thị:

- Entry: `router`
- Điểm vào: `router`
- Conditional edges: `router -> rag_node` or `router -> tool_node`
- Cạnh có điều kiện: `router -> rag_node` hoặc `router -> tool_node`
- Both `rag_node` and `tool_node` go to `END`.
- Cả `rag_node` và `tool_node` đều đi đến `END`.


In [ ]:
def router_edge(state: AgentState) -> str:
    # Hàm quyết định cạnh điều hướng từ router
    if state.route == 'rag':
        return 'rag_node'
    elif state.route == 'tool':
        return 'tool_node'
    return 'rag_node'  # Mặc định là rag_node

# Xây dựng đồ thị LangGraph
graph_builder = StateGraph(AgentState)
graph_builder.add_node('router', router_node)  # Thêm nút router
graph_builder.add_node('rag_node', rag_node)  # Thêm nút RAG
graph_builder.add_node('tool_node', tool_node)  # Thêm nút Tool
graph_builder.set_entry_point('router')  # Đặt điểm vào là router
graph_builder.add_conditional_edges('router', router_edge)  # Thêm cạnh có điều kiện từ router
graph_builder.add_edge('rag_node', END)  # RAG node kết thúc
graph_builder.add_edge('tool_node', END)  # Tool node kết thúc

app = graph_builder.compile()
print('✅ LangGraph graph compiled (once you define all components).')
print('✅ Đồ thị LangGraph đã được biên dịch (sau khi bạn định nghĩa tất cả các thành phần).')


## 12. Chat Loop – Test Your Agent
## 12. Vòng lặp Chat – Kiểm tra Agent của bạn

Finally, we define a simple console loop to interact with the agent.
Cuối cùng, chúng ta định nghĩa một vòng lặp console đơn giản để tương tác với agent.
Run this only **after** you have implemented all TODOs above.
Chạy điều này chỉ **sau khi** bạn đã triển khai tất cả các TODO ở trên.


In [ ]:
def chat_loop():
    # Vòng lặp chat để tương tác với agent
    print('🔹 Personal Study Copilot – type "exit" or "quit" to stop.')
    print('🔹 Personal Study Copilot – gõ "exit" hoặc "quit" để dừng.')
    while True:
        user_input = input('\nYou: ').strip()
        if user_input.lower() in ['exit', 'quit']:
            print('👋 Goodbye!')
            print('👋 Tạm biệt!')
            break
        # Tạo state mới và gọi app
        state = AgentState(user_input=user_input, messages=[])
        _ = app.invoke(state)

# Uncomment after everything is implemented:
# Bỏ comment sau khi mọi thứ đã được triển khai:
# chat_loop()


## 13. What to Submit
## 13. Những gì cần nộp

- This completed notebook (.ipynb).
- Notebook đã hoàn thành này (.ipynb).
- Screenshots or short video of:
- Ảnh chụp màn hình hoặc video ngắn về:
  - RAG answering study questions from your documents.
  - RAG trả lời câu hỏi học tập từ tài liệu của bạn.
  - Tool answering a math query.
  - Tool trả lời một truy vấn toán học.
- Short README / last markdown cell describing:
- README ngắn / cell markdown cuối cùng mô tả:
  - Which open‑source models you used.
  - Các mô hình mã nguồn mở bạn đã sử dụng.
  - Any limitations / issues.
  - Bất kỳ hạn chế / vấn đề nào.
